# Filtros

Transformar el sonido eliminando o atenuando frecuencias de manera selectiva

- Low pass LP (paso bajo): eliminan frecuencias altas (dejan pasar graves)

- Hihg pass HP (paso alto): eliminan frecuencias baja (dejan pasar agudos)

- Band pass BP (paso banda): pasa una porción dentro de una banda centrada en una frecuencia; elimina componentes fuera de esa banda.

- Band reject BR: al revés, descarta las frecuencias dentro de la banda.

- All pass AP: pasan todas, pero altera la fase de la señal de entrada.

![image](tipos_filtros.png)

**Frecuencia(s) de corte** (*cutoff frequency*): frecuencia a partir de la cual actúa el filtro.




En abstracto, un filtro digital transforma un conjunto de muestras

$$x[0,\ldots] \leadsto y[0,\ldots]$$ 

donde:

$$y[n]=\sum_{i=0}^N b_i*x[n-i] - a_i*y[n-i]$$

El filtro más simple

$$y[n] = x[n-1]+x[n]$$


# Contexto para probar los filtros

In [ ]:
%%writefile consts.py

import sounddevice as sd
import soundfile as sf
import numpy as np
import librosa
from tkinter import *

# utilizamos una frecuencia de muestreo de 8000 Hz para ver mejor el efecto del filtro
CHUNK, SRATE = 1204, 8000


### Clase Player

Generador de señal que toma la entrada de un archivo de audio

- Resampleamos a SRATE, sea cual sea el srate del audio leído: unificamos al SRATE del proyecto

In [ ]:
%%writefile player.py

from consts import *

class Player:
    def __init__(self, file):
        # leemos archivo de audio
        data, srate = sf.read(file,dtype=np.float32)       
        # resampleamos a SRATE
        self.data = librosa.resample(data, orig_sr=srate, target_sr=SRATE)     
        self.finished = False
        self.frame = 0

    def next(self):
        if self.frame+CHUNK>len(self.data): # si no queda suficiente data, rellenamos con ceros
            self.finished = True
            return np.pad(self.data[self.frame:],(0,CHUNK-len(self.data[self.frame:])),mode='constant')
        else:
            ret = self.data[self.frame:self.frame+CHUNK]
            self.frame += CHUNK
            return ret

    def isFinished(self):
        return self.finished
    
    def sRate(self):
        return self.SRATE

## Un filtro sencillo 

Implementado como procesador de señal:

- Recibe una señal (que tiene método *next*)

- Implementa su propio método *next* que devuelve audio procesado (filtrado)

In [ ]:
%%writefile filter.py

from consts import *

class Filter:
    def __init__(self,signal):
        self.signal = signal
        self.mem = 0
        # por defecto inactivo
        self.act = False

    def next(self):
        data = self.signal.next()
        if self.act:
            data[1:]=0.5*(data[0:-1]+data[1:]) # media entre cada muestra y la siguiente
            data[0] = 0.5*(self.mem+data[0]) # la primera muestra se calcula por separado, utilizando la última del bloque anterior
        self.mem = data[-1] # actualizamos memo con ultima muestra
        return data

    def activate(self):
        self.act = True

    def deactivate(self):
        self.act = False    

    def isActive(self):
        return self.act



## Hebra de soundDevice (con callback)

In [1]:
from consts import *

# stream de salida con callBack
input = None
def callback(outdata, frames, time, status):
    if status: print(status)
    if input:    
        s = input.next()
        s = np.float32(s)
    else:
        s = np.zeros(CHUNK,dtype=np.float32)
    outdata[:] = s.reshape(-1, 1)



# Recogida de señal, enrutado y control con TkInter

In [ ]:
from consts import *
from player import Player
from filter import Filter

# stream de salida con callBack
stream = sd.OutputStream(samplerate=SRATE, callback=callback, blocksize=CHUNK)
stream.start()


signal = Player('tormenta.wav')
signalFiltered = Filter(signal)
input = signalFiltered


# inicialización de la ventanas 
tk=Tk()

# Caja de texto
text = Text(tk,height=6,width=60)
text.pack(side=BOTTOM)
text.insert(INSERT,"Press 'F/f' to activate/deactivate filter\n")

# call back para la pulsación de teclas
def key_down(event):
    if event.char=='F': 
        text.insert(INSERT,f'\nFilter active  ')
        signalFiltered.activate()
    elif event.char=='f': 
        text.insert(INSERT,f'\nFilter bypassed  ')
        signalFiltered.deactivate()

# enlace de la pulsación de teclas con la función key_down
text.bind('<Key>', key_down)


# arrancamos todo!!
tk.mainloop()
# ejecución bloqueada hasta que se cierre ventana


stream.stop()       
stream.close()


In [ ]:
stream.close()

# Qué hace exactamente este filtro?

Cómo se comporta este filtro con una señal dada?
- más fácil: cómo se comporta para una frecuencia dada?

![image](lp_in-out.png)

-   Incrementa la amplitud hasta $\sqrt{2}$

-   Desplaza la fase en $-2*\pi/8$

Para profundizar: *Introduction to Digital Filters with Audio Applications*, by Julius O.
Smith III, (September 2007 Edition).



## Después se estudia cómo se comporta para *cada una de las frecuencias*

Analizando este filtro para cada seno ($f_s$ = frecuencia de muestreo)

![image](lp_response.png)

Para un análisis matemático detallado:

<https://ccrma.stanford.edu/~jos/filters/Mathematical_Sine_Wave_Analysis.html>



# Cómo transforma (gráficamente) la señal

Aplicamos este filtro LP a la tormenta:


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

data, SRATE = sf.read('tormenta.wav',dtype=np.float32)

#%% filtrado
CHUNK = 100 # solo para dibujar

nBloque = 0
original = data[nBloque*CHUNK:(nBloque+1)*CHUNK]
filtrada  = np.copy(original)

# varias pasadas de filtro para exagerar el efecto
N=10
for j in range(N):
    filtrada[1:] = 0.5*(filtrada[0:CHUNK-1]+filtrada[1:CHUNK])



#%% dibujamos original y filtrada
plt.figure(figsize=(20,10))
plt.plot(original)

plt.figure(figsize=(20,10))
plt.plot(filtrada)


# FIR y IIR

- El filtro anterior es un filtro FIR (*finite impulse response*). En general, un FIR es:

  $$y[n] = b_0 * x[n] + b_1 * x[n-1] + \ldots + b_* x[n-N]$$ 

  $x$: entrada; $y$ salida; $N$ orden del filtro; $b_i$ coeficientes del filtro

- Los filtros IIR (*infinite impulse response*) se definen como: 

  $$y[n] = \quad b_0 * x[n] + b_1 * x[n-1] + \ldots + b_N* x[n-N]\\
       \qquad\qquad\qquad -a_1 *y[n-1] - a_2*y[n-2] -\ldots - a_M*y[n-M]
    $$

**Diseño de filtros** cálculo de coeficientes $a_i,b_i$ para conseguir salida correspondiente: 
- frecuencia de corte, fase, amplitud
- y otras propiedades: linealidad, estabilidad, etc.

(Ampliamente estudiado en cursos de DSP)

# Un filtro IIR fácil

Un filtro LP un poco más sofisticado que el anterior:

$$y[n] = \alpha *y[n-1] + (1-\alpha)*x[n]$$

El coeficiente $\alpha$ determina la frecuencia de corte

**Importante:** ahora sí hay que hacer un bucle!! En una asignación como la de arriba, numpy

-  calcularía todo el lado derecho y luego lo asignaría al lado izdo ($y[n]$)... no es lo que queremos

No se puede hacer recursión en numpy de esta manera


In [ ]:
# Experimento: numpy primero calcula todo el lado dcho y luego asigna al izdo
b = np.arange(10,dtype=np.float32)
alpha =0.5

print('Experimento: ') # no funciona como queremos
b[1:] = b[0:-1] + alpha*(b[1:]-b[0:-1]) 
b[0]  = 0 + alpha*(b[0]-0)

# versión correcta
c = np.arange(10,dtype=np.float32)
c[0] = 0 + alpha * (c[0]-0)
for i in range(1,len(c)):
    c[i] = c[i-1] + alpha * (c[i]-c[i-1])

# comparamos resultados
print(b)
print(c)

In [5]:
%%writefile filterIIR.py

from consts import *

class FilterIIR:
    def __init__(self,signal,alpha,step=0.01):
        self.signal = signal
        self.mem = 0
        self.alpha = alpha
        self.step = step
        # por defecto inactivo
        self.act = False

    def next(self):
        data = self.signal.next()
        if self.act:
            data[0] = self.mem + self.alpha * (data[0]-self.mem)
            for i in range(1,CHUNK):
                data[i] = data[i-1] + self.alpha * (data[i]-data[i-1])
            self.mem = data[CHUNK-1]
        self.mem = data[-1] # actualizamos memo con ultima muestra
        return data

    def activate(self):
        self.act = True

    def deactivate(self):
        self.act = False    

    def isActive(self):
        return self.act

    def upAlpha(self):
        self.alpha = min(2.0,max(0.1,self.alpha+self.step))

    def downAlpha(self):
        self.alpha = min(2.0,max(0.1,self.alpha-self.step))

Overwriting filterIIR.py


In [2]:
from consts import *
from player import Player
from filterIIR import FilterIIR


# stream de salida con callBack
input = None
def callback(outdata, frames, time, status):
    if status: print(status)
    if input:    
        s = input.next()
        s = np.float32(s)
    else:
        s = np.zeros(CHUNK,dtype=np.float32)
    outdata[:] = s.reshape(-1, 1)

# stream de salida con callBack
stream = sd.OutputStream(samplerate=SRATE, callback=callback, blocksize=CHUNK)
stream.start()


signal = Player('tormenta.wav')
signalFiltered = FilterIIR(signal,alpha=0.5,step=0.05)
input = signalFiltered


# inicialización de la ventanas 
tk=Tk()

# Caja de texto
text = Text(tk,height=6,width=60)
text.pack(side=BOTTOM)
text.insert(INSERT,"Press 'F/f' to activate/deactivate filter\n")
text.insert(INSERT,"[A/a] sube baja alpha")


# call back para la pulsación de teclas
def key_down(event):
    if event.char=='F': 
        signalFiltered.activate()
    elif event.char=='f': 
        signalFiltered.deactivate()
    elif event.char=='A':
        signalFiltered.upAlpha()        
    elif event.char=='a':  
        signalFiltered.downAlpha()

    act = "active" if signalFiltered.isActive() else "bypassed"
    text.insert(INSERT,f'\nFilter {act}  Alpha: {signalFiltered.alpha}')        

# enlace de la pulsación de teclas con la función key_down
text.bind('<Key>', key_down)


# arrancamos todo!!
tk.mainloop()
# ejecución bloqueada hasta que se cierre ventana


stream.stop()       
stream.close()


ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred


output underflow
output underflow
output underflow
output underflow
output underflow


-   Qué ocurre con `alpha = 1`?
-   Y con `alpha = 0, alpha = 2`?

# Cómo hacemos un filtro HP?

Solo hemos visto filtros paso bajo (LP)... cómo hacemos uno HP (paso alto)?

Idea, dada una señal $x[n]$

-   Obtener la señal filtrada $y[n]$ mediante un filtro LP

-   Restar a la señal original $x[n]$ la señal filtrada $y[n]$

$$z[n]=x[n]-y[n]$$

## Cómo determinamos la frecuencia de corte?

En función de `alpha`:

$$alpha = e^{-2*\pi*freq/SRATE}$$



In [ ]:

# Implementación de un filtro paso bajo con una frecuencia de corte variable (IDEA)


print("[F] frecuencia de corte +100")
print("[f] frecuencia de corte -100")
print("[l] filtro lp")
print("[h] filtro hp")

freq = 1000
prev = 0
filter = "lp"


# en el filtro
'''
    bloque = data[nBloque*CHUNK : (nBloque+1)*CHUNK]
        
    alpha = np.exp(-2*np.pi*freq / SRATE)    

    # filtro paso bajo
    if (filter=='lp'):
        bloque[0] = alpha * prev + (1-alpha) * bloque[0]
        for i in range(1,CHUNK):         
            bloque[i] = alpha * bloque[i-1] + (1-alpha) * bloque[i]            
    # filtro paso alto (diferencia entre señal original y paso bajo)
    elif (filter=='hp'):
        bloque[0] = bloque[0] - alpha * prev + (1-alpha) * bloque[0]
        for i in range(1,CHUNK):
            bloque[i] = bloque[i] - (alpha * bloque[i-1] + (1-alpha) * bloque[i])

    prev = bloque[CHUNK-1]
'''

# Más sobre filtros

-   *Introduction to Digital Filters with Audio Applications*, Julius O. Smith III, (September 2007 Edition). (online)

-   *The Scientist and Engineer's Guide to Digital Signal Processing*, Second Edition, Steven W. Smith, 1999,
    (www.DSPguide.com)

-   *Real sound synthesis for interactive applications*, Perry  Cook, 2002.

DSP en NumPy:

<https://docs.scipy.org/doc/scipy/reference/signal.html>

También ofrece:

-  **Correlación**: *medida de la similitud entre señales* (análisis de espectro)

-  **Convolución**: *efecto de aplicar una señal a otra* (efectos)

-  **FFT**: transformada rápida de Fourier $\leadsto$ eficiencia en las operaciones (tiempo real)
